In [2]:
import os
import cv2
import numpy as np

def load_images_from_directories(real_images_dir, fake_images_dir):
    data = []  
    labels = []  
    try:
        for image_file in os.listdir(real_images_dir):  
            if image_file.endswith(('.jpg', '.jpeg', '.png', '.bmp')): 
                image_path = os.path.join(real_images_dir, image_file) 
                image = cv2.imread(image_path)  
               
                image = cv2.resize(image, (224, 224))
                
                image = image.astype("float") / 255.0
                data.append(image)  
                labels.append("real")  

        for image_file in os.listdir(fake_images_dir):  
            if image_file.endswith(('.jpg', '.jpeg', '.png', '.bmp')):  
                image_path = os.path.join(fake_images_dir, image_file)  
                image = cv2.imread(image_path)  
                
                image = cv2.resize(image, (224, 224))
                image = image.astype("float") / 255.0
                data.append(image)  
                labels.append("fake")  

        return np.array(data), np.array(labels)  

    except FileNotFoundError:
        print("One or both directories not found. Please provide the correct paths.")

real_images_dir = r"C:\Users\SOHAN GHOSH\Downloads\DeepFake_Detection\DeepFake_Detection\Real"  
fake_images_dir = r"C:\Users\SOHAN GHOSH\Downloads\DeepFake_Detection\DeepFake_Detection\Fake" 
data, labels = load_images_from_directories(real_images_dir, fake_images_dir)

if data is not None and labels is not None:
    print("Dataset loaded successfully.")
    print("Number of images:", len(data))
    print("Shape of X (images):", data.shape)
    print("Number of labels:", len(labels))
    print("Labels:", np.unique(labels))
else:
    print("Failed to load the dataset.")


Dataset loaded successfully.
Number of images: 2041
Shape of X (images): (2041, 224, 224, 3)
Number of labels: 2041
Labels: ['fake' 'real']


In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)


Shape of X_train: (1632, 224, 224, 3)
Shape of X_test: (409, 224, 224, 3)
Shape of y_train: (1632,)
Shape of y_test: (409,)


In [4]:
import tensorflow as tf
from tensorflow.keras import layers, models

def create_cnn_model(input_shape):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(1, activation='sigmoid')  
    ])

    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    return model

input_shape = (224, 224, 3)
model = create_cnn_model(input_shape)
model.summary()

history = model.fit(X_train, (y_train == "fake").astype(int), 
                    epochs=10,
                    batch_size=32,
                    validation_data=(X_test, (y_test == "fake").astype(int)))


C:\Users\SOHAN GHOSH\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,169,089 (42.61 MB)

 Trainable params: 11,169,089 (42.61 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
51/51 ━━━━━━━━━━━━━━━━━━━━ 76s 1s/step - accuracy: 0.4677 - loss: 1.0690 - val_accuracy: 0.5110 - val_loss: 0.6915
Epoch 2/10
51/51 ━━━━━━━━━━━━━━━━━━━━ 66s 1s/step - accuracy: 0.5295 - loss: 0.6939 - val_accuracy: 0.5501 - val_loss: 0.6754
Epoch 3/10
51/51 ━━━━━━━━━━━━━━━━━━━━ 67s 1s/step - accuracy: 0.5942 - loss: 0.6746 - val_accuracy: 0.5892 - val_loss: 0.6603
Epoch 4/10
51/51 ━━━━━━━━━━━━━━━━━━━━ 61s 1s/step - accuracy: 0.6707 - loss: 0.6319 - val_accuracy: 0.5648 - val_loss: 0.6684
Epoch 5/10
51/51 ━━━━━━━━━━━━━━━━━━━━ 58s 1s/step - accuracy: 0.7061 - loss: 0.5689 - val_accuracy: 0.5575 - val_loss: 0.7018
Epoch 6/10
51/51 ━━━━━━━━━━━━━━━━━━━━ 55s 1s/step - accuracy: 0.7635 - loss: 0.4889 - val_accuracy: 0.5501 - val_loss: 0.7450
Epoch 7/10
51/51 ━━━━━━━━━━━━━━━━━━━━ 81s 2s/step - accuracy: 0.8259 - loss: 0.3880 - val_accuracy: 0.5501 - val_loss: 0.9682
Epoch 8/10
51/51 ━━━━━━━━━━━━━━━━━━━━ 58s 1s/step - accuracy: 0.8892 - loss: 0.2604 - val_accuracy: 0.5208 - val_loss:

In [5]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 33,507,269 (127.82 MB)

 Trainable params: 11,169,089 (42.61 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 22,338,180 (85.21 MB)

In [6]:

loss, accuracy = model.evaluate(X_test, (y_test == "fake").astype(int))


print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 409ms/step - accuracy: 0.5004 - loss: 2.4428
Test Loss: 2.3310811519622803
Test Accuracy: 0.5403422713279724


In [7]:
import numpy as np
import cv2
import requests

def classify_image(image_path_or_url):
    try:
        # Check if the provided input is a URL
        if image_path_or_url.startswith('http://') or image_path_or_url.startswith('https://'):
            # Download the image from the URL
            response = requests.get(image_path_or_url)
            # Ensure the request was successful
            if response.status_code == 200:
                # Convert image content to numpy array
                image_array = np.asarray(bytearray(response.content), dtype=np.uint8)
                # Read the image using OpenCV
                user_image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)
                # Check if the image is valid
                if user_image is None:
                    print("Error: Unable to decode image")
                    return None
            else:
                print("Error: Unable to download image from URL")
                return None
        else:
            # If not a URL, assume it's a local file path
            image_path = image_path_or_url
            # Read the image using OpenCV
            user_image = cv2.imread(image_path)
            # Check if the image is valid
            if user_image is None:
                print("Error: Unable to load image at path:", image_path)
                return None
        
        # Resize image to the required dimensions (224x224)
        user_image = cv2.resize(user_image, (224, 224))
        # Normalize pixel values to be between 0 and 1
        user_image = user_image.astype('float32') / 255.0
        user_image = np.expand_dims(user_image, axis=0)
        # Pass the preprocessed image through the trained model
        predicted_label = model.predict(user_image)
        # Convert predicted probability to class label
        if predicted_label >= 0.5:
            return "Fake"
        else:
            return "Real"
    except Exception as e:
        print("Error:", e)
        return None

# Example usage:
image_path_or_url = r"https://miro.medium.com/v2/resize:fit:828/format:webp/0*0zHFF42lGRCCkGSg.jpg"  # Local file path or online image URL
predicted_label = classify_image(image_path_or_url)
if predicted_label is not None:
    print("Predicted label:", predicted_label)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 382ms/step
Predicted label: Fake


In [8]:
import pickle
pickle.dump(model, open('model.pkl','wb'))

In [9]:
model.save("model/deepfake_model.h5")